In [1]:
!pip install -Uq langchain langchain_community langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.3 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
import os

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')


In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash',verbose=True)

llm.invoke("Hi")

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-abf64f17-d0d0-41ca-8f2f-5692eb6add41-0', usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}})

In [6]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

llm_with_tools = llm.bind_tools([multiply])

In [7]:
tools_list=[multiply]
model_with_tools = llm.bind_tools(tools_list)

In [8]:
result = llm_with_tools.invoke("Hey how are you!")
result

AIMessage(content="I'm doing well, thank you for asking! How are you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-c9f3222e-6c70-4a07-9954-0d36c06f8677-0', usage_metadata={'input_tokens': 68, 'output_tokens': 17, 'total_tokens': 85, 'input_token_details': {'cache_read': 0}})

In [9]:
result = llm_with_tools.invoke("What is 2 multiplied by 3?")
result


AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 2.0, "b": 3.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-470c5ed0-460f-4b53-8c9d-d370457715e8-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2.0, 'b': 3.0}, 'id': 'a61346b1-6488-4000-8bf9-e5b9e82ffb04', 'type': 'tool_call'}], usage_metadata={'input_tokens': 72, 'output_tokens': 3, 'total_tokens': 75, 'input_token_details': {'cache_read': 0}})

In [14]:
def add_numbers(x:int,y:int):
  """Add two numbers x and y.
  Args:
    x:1st int:
    y:2nd int
  """
  return x + y
llm=llm.bind_tools([add_numbers])
res=llm.invoke("what is 10 plus 80")
res

AIMessage(content='', additional_kwargs={'function_call': {'name': 'add_numbers', 'arguments': '{"y": 80.0, "x": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-9eb4bc16-1bf8-4965-aadc-23fc754af59d-0', tool_calls=[{'name': 'add_numbers', 'args': {'y': 80.0, 'x': 10.0}, 'id': '1ea22c62-588b-44ff-8cdc-e5a5aafa5dc7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 78, 'output_tokens': 5, 'total_tokens': 83, 'input_token_details': {'cache_read': 0}})

In [16]:
from langchain.tools import Tool

# Example tool for addition
def add_numbers(a, b):
    return a + b

math_tool = Tool(
    name="AdditionTool",
    func=lambda x: f"The result is: {add_numbers(*map(int, x.split()))}",
    description="Adds two numbers. Input should be two integers separated by a space."
)
math_tool

Tool(name='AdditionTool', description='Adds two numbers. Input should be two integers separated by a space.', func=<function <lambda> at 0x783998b17c40>)

In [27]:
from langchain.agents import initialize_agent, AgentType

# List of tools
tools = [math_tool]

# Initialize the agent
agent = initialize_agent(
    tools=tools,                        # Provide the tools
    llm=llm,                            # LLM for fallback
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Agent type
    verbose=False                       # Enable debugging output
)


In [28]:
# Query the agent
response1 = agent.run("What is 10 + 20 -10/2?")  # Uses the tool
print(response1)



25
